In [1]:
import cv2
import os
import numpy as np
import os
from tensorflow import keras
import threading
from IPython.display import clear_output
import time

In [2]:
def resize_image(img):
    #scale_percent = 10 # percent of original size
    #width = int(img.shape[1] * scale_percent / 100)
    #height = int(img.shape[0] * scale_percent / 100)
    #dim = (width, height)
    
    # resize image
    resized = cv2.resize(img, (80,80), interpolation = cv2.INTER_AREA)
    
    print('Resized Dimensions : ',resized.shape)
    
    #cv2.imshow("Resized image", resized)
    return resized

In [3]:
# validating images
tick = cv2.imread("images/tick.png")
cross = cv2.imread("images/cross.png")

tick = resize_image(tick)
cross = resize_image(cross)

Resized Dimensions :  (80, 80, 3)
Resized Dimensions :  (80, 80, 3)


In [4]:
def get_output_layers(net):
    layer_names = net.getLayerNames() 
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    return output_layers

def draw_bounding_box(img, confidence, x, y, x_plus_w, y_plus_h):
    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), (0,255,0), 2)
    cv2.putText(img, "Person", (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)

In [5]:
net = cv2.dnn.readNet('yolo/yolov3.weights', 'yolo/object-detection-opencv/yolov3.cfg')

In [6]:
cascPathface = os.path.dirname(
    cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
#cascPatheyes = os.path.dirname(
    #cv2.__file__) + "/data/haarcascade_eye_tree_eyeglasses.xml"

In [7]:
faceCascade = cv2.CascadeClassifier(cascPathface)
#eyeCascade = cv2.CascadeClassifier(cascPatheyes)

In [8]:
model = keras.models.load_model('face_detection/my_model.h5')

In [9]:
def detectPerson(frame):
    Width = frame.shape[1]
    Height = frame.shape[0]
    scale = 0.00392
    blob = cv2.dnn.blobFromImage(frame, scale, (416,416), (0,0,0), True, crop=False)

    net.setInput(blob)

    outs = net.forward(get_output_layers(net))
    
    class_ids = []
    confidences = []
    boxes = []
    conf_threshold = 0.5
    nms_threshold = 0.4

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * Width)
                center_y = int(detection[1] * Height)
                w = int(detection[2] * Width)
                h = int(detection[3] * Height)
                x = center_x - w / 2
                y = center_y - h / 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])


    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

    for i in indices:
        i = i[0]
        box = boxes[i]
        x = box[0]
        y = box[1]
        w = box[2]
        h = box[3]
        
        draw_bounding_box(frame, confidences[i], round(x), round(y), round(x+w), round(y+h))

    # return frame

In [10]:
def detectFace(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)
    #print("No. of faces detected: "+str(len(faces)))
    
    if(len(faces)>0):
        face_present=True    

        for (x,y,w,h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h),(0,255,0), 2)
            cv2.putText(frame, 'Face', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
            faceROI = frame[y:y+h,x:x+w]

In [11]:
def detectMask(frame, start, tick, cross):
    final_image = cv2.resize(frame,(224,224))
    final_image = np.expand_dims(final_image,axis = 0)
    final_image = final_image/255.0
    
    predictions = model.predict(final_image)
    end=time.time()
    clear_output(wait=True)
    print("FPS: ", 1/(end-start))
    alpha = 0.4
    if(predictions>0.1):
        added_image = cv2.addWeighted(frame[0:80,0:80,:],alpha,cross,1-alpha,0)
        frame[0:80,0:80,:] = cross
        print("No mask!")
    else:
        added_image = cv2.addWeighted(frame[0:80,0:80,:],alpha,tick,1-alpha,0)
        frame[0:80,0:80,:] = tick
        print("Mask present!")

In [ ]:
# determining decison parameters
face_present=False
person_present=False
mask_present=False

start=0

video_capture = cv2.VideoCapture("test.mp4")
frame_width = int(video_capture.get(3)) 
frame_height = int(video_capture.get(4)) 
   
size = (frame_width, frame_height) 

result = cv2.VideoWriter('filename.avi',  
                         cv2.VideoWriter_fourcc(*'MJPG'), 
                         10, size) 
while True:
    ret, frame = video_capture.read()
    
    t1 = threading.Thread(target=detectPerson, args=(frame,))
    t2 = threading.Thread(target=detectFace, args=(frame,))
    t3 = threading.Thread(target=detectMask, args=(frame,start,tick,cross))
    
    ########################################### person detection ###########################################
    
    t1.start()


    ########################################### face detection ###########################################
    
    t2.start()


    ########################################### Mask detection ########################################### 
    
    t3.start()




    t1.join()
    t2.join()
    t3.join()

    start=time.time()

    # Display the resulting frame
    #cv2.imshow('Video', frame)
    result.write(frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()

FPS:  11.39391173482416
Mask present!
